## https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions list

In [ ]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

In [ ]:
!unzip gtzan-dataset-music-genre-classification.zip

In [ ]:
import IPython.display as ipd
ipd.Audio('/content/Data/genres_original/hiphop/hiphop.00001.wav',rate=16000)

In [ ]:
#오디오 파일도 숫자로 변형해서 분석
#y 소리가가 떨리는 세기를 시간순서대로 나열열
#sr (sampliing rate 1초당 샘플의 개수 )
import librosa
y,sr = librosa.load('Data/genres_original/reggae/reggae.00036.wav')

print(y) #소리가 떨리는 진폭의 배열
print(len(y))
print(sr) #초당 샘플의 개수 많을수록 음질깨끗??
len(y)/sr

In [ ]:
import IPython.display as ipd

ipd.Audio(y, rate=sr)

In [ ]:
import matplotlib.pyplot as plt
import librosa.display

plt.figure(figsize=(16,6))
librosa.display.waveshow(y=y,sr=sr)
plt.show()

In [ ]:
#시간 데이터를 주파수영역 데이터로 변경 분석이 더 잘됨
#STFT(Short Time Fourier Transform)
#푸리에 변환 : 입력 신호를 다양한 주파수를 가지는 주기함수들로 분해하는 것
#1. n_fft : length of the windowed signal after padding with zeros.
#            한 번 fft를 해 줄 만큼의 sequence 길이
#2. hop_length : window 간의 거리
#3. win_length : window 길이..
import numpy as np

# win_length 는 음성을 작은 조각으로 자를때 작은 조각의 크기입니다.
# hop_length 는 음성을 작은 조각으로 자를때 자르는 간격을 의미합니다.
# n_mels 는 적용할 mel filter의 개수입니다.

D = np.abs(librosa.stft(y,n_fft=2048,hop_length=512))

plt.figure(figsize=(16,6))
plt.plot(D)
plt.show()

In [ ]:
#spectogram
#시간에 따른 신호 주파수의 스펙트럼 그래프

DB = librosa.amplitude_to_db(D,ref=np.max)

plt.figure(figsize=(16,6))
librosa.display.specshow(DB,sr=sr,hop_length=512, x_axis='time' , y_axis='log')
plt.show()

In [ ]:
#알아먹기기 힘든 스펙토그램을 y축을을 mel scale로로 변환하여 좀더 알아먹기편하게 변형
#주파수의 단위를 다음 공식에 따라 멜 단위(Mel unit)로 바꾼 스펙트럼을 말한다.
#- Mel-scale
#Mel-scale은 이러한 pitch에서 발견한 사람의 음을 인지하는 기준(threshold)을 반영한 scale 변환 함수이다.

S = librosa.feature.melspectrogram(y=y,sr=sr)
S_DB = librosa.amplitude_to_db(S,ref=np.max)

plt.figure(figsize=(16,6))
librosa.display.specshow(S_DB,sr=sr,hop_length=512, x_axis='time' , y_axis='log')
plt.show()

In [ ]:
spec = librosa.feature.melspectrogram(
    y=y,
    sr=sr,
    n_fft=2048,
    hop_length=512,
    win_length=None,
    window='hann',
    center=True,
    pad_mode='reflect',
    power=2.0,
    n_mels=128
)

In [ ]:
spec.shape

## music_mel로 음향 만들기

In [ ]:
import numpy as np
np.save('music_mel.npy',spec)

In [ ]:
spec_new = np.load('music_mel.npy')
spec_new

In [ ]:
res = librosa.feature.inverse.mel_to_audio(
    spec_new,
    sr=sr,
    n_fft=2048,
    hop_length=512,
    win_length=None,
    window='hann',
    center=True,
    pad_mode='reflect',
    power=2.0,
    n_iter=128
)

In [ ]:
import soundfile
soundfile.write('mel_test.wav',res,22050)

In [ ]:
import IPython.display as ipd
ipd.Audio('mel_test.wav')

## 오디오 특성 추출

In [ ]:
#음악의 BPM 추출
#BPM은 Beat Per Minute의 약자로써 분당 박자 수
tempo,_ = librosa.beat.beat_track(y=y,sr=sr)
tempo

In [ ]:
#음파가가 양에서서 음으로 음에서서 양으로로 변환하는값
zero_crossings = librosa.zero_crossings(y,pad=False)

len(zero_crossings)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(y[3000:3500])
plt.show()

In [ ]:
#Harmonics : 사람의 귀로 구분할 수 없는 특징들
#Percussives : 리듬과 감정을 나타내는 충격파

y_harm, y_perc = librosa.effects.hpss(y)
print(y_harm)
print(y_perc)

plt.figure(figsize=(16,6))
plt.plot(y_harm,color='b')
plt.plot(y_perc,color='r')
plt.show()

In [ ]:
#Spectral Centroid
#소리를 주파수 표현했을때 , 주파수의 가중평균을 계산하여 소리의 무게중심을 알려줌
#블루스음악 무게중심 가운데 , 메탈 마지막
import sklearn

spectral_centroids = librosa.feature.spectral_centroid(y=y,sr=sr)[0]

frames = range(len(spectral_centroids))
print(frames)
t = librosa.frames_to_time(frames)

spec_norm = sklearn.preprocessing.minmax_scale(spectral_centroids,axis=0)

plt.figure(figsize=(16,6))
librosa.display.waveshow(y=y,sr=sr,alpha=0.5,color='b')
plt.plot(t,spec_norm,color='r')
plt.show()

In [ ]:
spec_norm

In [ ]:
#spectral rolloff
#신호의 모양을 측정 , 총 스펙트럴 에너지중 낮은 주파수에 얼마나 많이 집중되는지

spectral_rolloff = librosa.feature.spectral_rolloff(y=y,sr=sr)[0]
print(spectral_rolloff)

In [ ]:
import sklearn

def normalize(x,axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)
#MFCCs 오디오 특징들의 집합
#음악의 장르를 정하는 것은 사람의 청각
#사람의 청각 구조를 반영하여 음성 정보 추출
#MFCC란?
#MFCC는 오디오 신호에서 추출할 수 있는 feature로, 소리의 고유한 특징을 나타내는 수치입니다.
#주로 음성 인식, 화자 인식, 음성 합성, 음악 장르 분류 등 오디오 도메인의 문제를 해결하는 데 사용됩니다.

mfccs = librosa.feature.mfcc(y=y,sr=sr)
mfccs = normalize(mfccs,axis=1)

print(mfccs.mean()) #평균
print(mfccs.var())  #분산

In [ ]:
#chroma frequencies
#크로마특징은 음악의 흥미롭고 강렬한 표현
#크로마는 인간 청각이 옥타브 차이가 나는 주파수를 가진 두음을 유사음으로 인지

chromagram = librosa.feature.chroma_stft(y=y,sr=sr, hop_length=512)
chromagram

In [ ]:
#위의 특징들을 컬럼에 넣어서 유사도 측정
import pandas as pd
df = pd.read_csv('./Data/features_3_sec.csv')
df.head()

In [ ]:
X = df.drop(columns=['filename','length','label'])
y = df['label']

scaler = sklearn.preprocessing.MinMaxScaler()
np_scaled = scaler.fit_transform(X)

X = pd.DataFrame(np_scaled, columns=X.columns)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)
print(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2222)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from xgboost import XGBClassifier
from sklearn import preprocessing

# Regression, Classification 문제를 모두 지원하며, 성능과 자원 효율이 좋아서, 인기 있게 사용되는 알고리즘
#n_estimators 예측까지
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
xgb.fit(X_train, y_train)

y_preds = xgb.predict(X_test)

print('Accuracy : ',accuracy_score(y_test,y_preds))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_preds)

plt.figure()
sns.heatmap(cm,annot=True ,
            xticklabels=['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock'],
            yticklabels=['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock'],
            )

In [ ]:
df_30 = pd.read_csv('./Data/features_30_sec.csv',index_col='filename')
df_30.head()

In [ ]:
labels = df_30.label

In [ ]:
df_30 = df_30.drop(columns=['length','label'])
df_30_scaled = sklearn.preprocessing.scale(df_30)
df_30 = pd.DataFrame(df_30_scaled,columns = df_30.columns)
df_30.head()

In [ ]:
from numba.core.errors import ConstantInferenceError
#코사인유사도
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(df_30)
sim_df = pd.DataFrame(similarity,index=labels.index, columns=labels.index)
sim_df.head()

In [ ]:
music = sim_df['blues.00000.wav'].sort_values(ascending=False)

In [ ]:
music.index[1]